<a href="https://colab.research.google.com/github/danielelbrecht/CAP5610-HW-2/blob/master/CAP5610_HW2_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement k-fold cross validation on final model, and compare validation accuracy to simple holdout validation

In [5]:
from keras.datasets import cifar10
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow.keras import layers, utils

Using TensorFlow backend.


In [0]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [0]:
# Preprocess data
train_images = train_images.reshape((50000, 32, 32, 3)) / 255.0
test_images = test_images.reshape((10000, 32, 32, 3)) / 255.0

train_labels_categorical = utils.to_categorical(train_labels, num_classes=10, dtype='float32')
test_labels_categorical = utils.to_categorical(test_labels, num_classes=10, dtype='float32')



In [0]:
#k-fold cross validation function

def kfold(k, model, data, labels):
  
  epochs = 20
  length = len(data)
  accuracy = 0
  model.save_weights('initial_weights')
  
  for i in range(k):
    
    model.load_weights('initial_weights')
    
    #Get validation and training splits from data set
    lower_bound = int(i*(length/k))
    upper_bound = int((i+1)*(length/k))

    train_data = np.concatenate((data[0:lower_bound], data[upper_bound:length]))
    val_data = data[lower_bound:upper_bound]
    
    train_labels = np.concatenate((labels[0:lower_bound], labels[upper_bound:length]))
    val_labels = labels[lower_bound:upper_bound]
    
    history = model.fit(train_data, 
                      train_labels, 
                      epochs=epochs,
                      validation_data=(val_data, val_labels))
    
    accuracy = accuracy + history.history['acc'][epochs-1]
    

  return accuracy / k
    

In [0]:
#Define model

model = tf.keras.Sequential()

# First convolutional module
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(32,32,3)))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))

# Second convolutional module
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))


# Fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
#Perform 5-fold cross validation

kfold(5, model, train_images, train_labels_categorical)


Consider using a TensorFlow optimizer from `tf.train`.
Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 165s 4ms/sample - loss: 1.5885 - acc: 0.4231 - val_loss: 1.2733 - val_acc: 0.5451
Epoch 2/20
40000/40000 [==============================] - 166s 4ms/sample - loss: 1.1875 - acc: 0.5819 - val_loss: 1.0366 - val_acc: 0.6412
Epoch 3/20
40000/40000 [==============================] - 166s 4ms/sample - loss: 1.0370 - acc: 0.6372 - val_loss: 1.0180 - val_acc: 0.6424
Epoch 4/20
40000/40000 [==============================] - 165s 4ms/sample - loss: 0.9329 - acc: 0.6755 - val_loss: 0.9946 - val_acc: 0.6503
Epoch 5/20
40000/40000 [==============================] - 166s 4ms/sample - loss: 0.8649 - acc: 0.6979 - val_loss: 0.8495 - val_acc: 0.7084
Epoch 6/20
40000/40000 [==============================] - 167s 4ms/sample - loss: 0.8132 - acc: 0.7175 - val_loss: 0.8542 - val_acc: 0.7089
Epoch 7/20
40000/40000 [==============================

0.7980599880218506

In [6]:
val_acc = [0.7216, 0.6917, 0.7118, 0.676, 0.7086]
print("Average validation accuracy for k-fold is %f" % np.mean(val_acc))

Average validation accuracy for k-fold is 0.701940
